# VAEGAN: Hybrid VAE-GAN Model

This notebook implements a hybrid VAEGAN model that combines:
- VAE as generator with 4-layer encoder/decoder
- WGAN critic for adversarial training
- Modified VAE loss with critic feedback


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import numpy as np
import logging
from collections import defaultdict

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(name)s | %(levelname)s | %(message)s'
)
logger = logging.getLogger('VAEGAN')
logger.info(f"Device: {device}")


In [ ]:
class DataLoaderModule:
    """Simple dataloader for MNIST and CIFAR10"""
    
    def __init__(self, batch_size=64):
        self.batch_size = batch_size
        self.logger = logging.getLogger('VAEGAN.DataLoader')
        
    def load_mnist(self):
        self.logger.info("Loading MNIST dataset")
        transform = transforms.Compose([
            transforms.Resize(64),
            transforms.ToTensor(),
            transforms.Normalize([0.5], [0.5])  # [-1, 1] range
        ])
        
        dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
        dataloader = torch.utils.data.DataLoader(dataset, batch_size=self.batch_size, shuffle=True)
        self.logger.info(f"MNIST loaded: {len(dataloader)} batches")
        return dataloader, 1, [64, 64]
    
    def load_cifar10(self):
        self.logger.info("Loading CIFAR10 dataset")
        transform = transforms.Compose([
            transforms.Resize(64),
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # [-1, 1] range
        ])
        
        dataset = datasets.CIFAR10('./data', train=True, download=True, transform=transform)
        dataloader = torch.utils.data.DataLoader(dataset, batch_size=self.batch_size, shuffle=True)
        self.logger.info(f"CIFAR10 loaded: {len(dataloader)} batches")
        return dataloader, 3, [64, 64]


In [ ]:
class VAEGenerator(nn.Module):
    """Dynamic VAE with 4-layer encoder and decoder based on latent_dim"""
    
    def __init__(self, input_channels=1, latent_dim=128, img_size=64):
        super().__init__()
        self.latent_dim = latent_dim
        self.img_size = img_size
        self.logger = logging.getLogger('VAEGAN.VAEGenerator')
        
        # Dynamic channel sizes based on latent_dim
        base_dim = latent_dim // 2  # latent_dim/2
        dim2 = latent_dim           # latent_dim
        dim3 = latent_dim * 2       # latent_dim*2  
        dim4 = latent_dim * 4       # latent_dim*4
        
        # 4-layer encoder - progressively increase channels
        self.encoder = nn.Sequential(
            nn.Conv2d(input_channels, base_dim, 4, 2, 1),  # 64x64 -> 32x32
            nn.BatchNorm2d(base_dim),
            nn.ReLU(),
            
            nn.Conv2d(base_dim, dim2, 4, 2, 1),  # 32x32 -> 16x16
            nn.BatchNorm2d(dim2),
            nn.ReLU(),
            
            nn.Conv2d(dim2, dim3, 4, 2, 1),  # 16x16 -> 8x8
            nn.BatchNorm2d(dim3),
            nn.ReLU(),
            
            nn.Conv2d(dim3, dim4, 4, 2, 1),  # 8x8 -> 4x4
            nn.BatchNorm2d(dim4),
            nn.ReLU(),
        )
        
        # Latent projection
        self.fc_mu = nn.Linear(dim4 * 4 * 4, latent_dim)
        self.fc_logvar = nn.Linear(dim4 * 4 * 4, latent_dim)
        self.fc_decode = nn.Linear(latent_dim, dim4 * 4 * 4)
        
        # 4-layer decoder - progressively decrease channels
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(dim4, dim3, 4, 2, 1),  # 4x4 -> 8x8
            nn.BatchNorm2d(dim3),
            nn.ReLU(),
            
            nn.ConvTranspose2d(dim3, dim2, 4, 2, 1),  # 8x8 -> 16x16
            nn.BatchNorm2d(dim2),
            nn.ReLU(),
            
            nn.ConvTranspose2d(dim2, base_dim, 4, 2, 1),  # 16x16 -> 32x32
            nn.BatchNorm2d(base_dim),
            nn.ReLU(),
            
            nn.ConvTranspose2d(base_dim, input_channels, 4, 2, 1),  # 32x32 -> 64x64
            nn.Tanh()
        )
        
        self.logger.info(f"Dynamic VAE Generator: channels={input_channels}, latent_dim={latent_dim}, dims=[{base_dim}, {dim2}, {dim3}, {dim4}]")
    
    def encode(self, x):
        h = self.encoder(x)
        h = h.view(h.size(0), -1)
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        return mu, logvar
    
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std
    
    def decode(self, z):
        h = self.fc_decode(z)
        h = h.view(h.size(0), self.latent_dim * 4, 4, 4)
        return self.decoder(h)
    
    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        recon = self.decode(z)
        return recon, mu, logvar


In [ ]:
class Critic(nn.Module):
    """Dynamic WGAN-GP Critic with gradient penalty"""
    
    def __init__(self, input_channels=1, latent_dim=128):
        super().__init__()
        self.logger = logging.getLogger('VAEGAN.Critic')
        
        # Dynamic channel sizes based on latent_dim
        base_dim = latent_dim // 2  # latent_dim/2
        dim2 = latent_dim           # latent_dim
        dim3 = latent_dim * 2       # latent_dim*2
        dim4 = latent_dim * 4       # latent_dim*4
        
        self.conv_layers = nn.Sequential(
            nn.Conv2d(input_channels, base_dim, 4, 2, 1),
            nn.LeakyReLU(0.2),
            
            nn.Conv2d(base_dim, dim2, 4, 2, 1),
            nn.BatchNorm2d(dim2),
            nn.LeakyReLU(0.2),
            
            nn.Conv2d(dim2, dim3, 4, 2, 1),
            nn.BatchNorm2d(dim3),
            nn.LeakyReLU(0.2),
            
            nn.Conv2d(dim3, dim4, 4, 2, 1),
            nn.BatchNorm2d(dim4),
            nn.LeakyReLU(0.2),
        )
        
        self.fc = nn.Linear(dim4 * 4 * 4, 1)
        self.logger.info(f"Dynamic Critic: channels={input_channels}, latent_dim={latent_dim}, dims=[{base_dim}, {dim2}, {dim3}, {dim4}]")
    
    def forward(self, x):
        h = self.conv_layers(x)
        h = h.view(h.size(0), -1)
        return self.fc(h).squeeze()
    
    def gradient_penalty(self, real_images, fake_images, device):
        """Calculate gradient penalty for WGAN-GP"""
        batch_size = real_images.size(0)
        
        # Random interpolation weight
        alpha = torch.rand(batch_size, 1, 1, 1, device=device)
        
        # Interpolated images
        interpolated = alpha * real_images + (1 - alpha) * fake_images
        interpolated.requires_grad_(True)
        
        # Critic scores for interpolated images
        scores = self.forward(interpolated)
        
        # Calculate gradients
        gradients = torch.autograd.grad(
            outputs=scores,
            inputs=interpolated,
            grad_outputs=torch.ones_like(scores),
            create_graph=True,
            retain_graph=True,
            only_inputs=True
        )[0]
        
        # Gradient penalty
        gradients = gradients.view(batch_size, -1)
        gradient_norm = gradients.norm(2, dim=1)
        penalty = torch.mean((gradient_norm - 1) ** 2)
        
        return penalty


In [ ]:
class VAEGAN(nn.Module):
    """Dynamic Hybrid VAEGAN model with VAE generator and WGAN-GP critic"""
    
    def __init__(self, input_channels=1, latent_dim=128, img_size=64, lambda_gp=10):
        super().__init__()
        self.logger = logging.getLogger('VAEGAN.Model')
        self.lambda_gp = lambda_gp
        
        # Dynamic models based on latent_dim
        self.generator = VAEGenerator(input_channels, latent_dim, img_size).to(device)
        self.critic = Critic(input_channels, latent_dim).to(device)
        
        self.logger.info(f"Dynamic VAEGAN initialized: latent_dim={latent_dim}, lambda_gp={lambda_gp}")
    
    def vae_loss(self, x, recon, mu, logvar, critic_score):
        """Modified VAE loss with critic feedback"""
        # Standard VAE losses
        recon_loss = F.mse_loss(recon, x, reduction='mean')
        kl_loss = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
        
        # Adversarial loss - generator wants high critic scores
        adv_loss = -torch.mean(critic_score)
        
        return recon_loss, kl_loss, adv_loss
    
    def critic_loss(self, real_score, fake_score, gp):
        """WGAN-GP critic loss with gradient penalty"""
        return torch.mean(fake_score) - torch.mean(real_score) + self.lambda_gp * gp
    
    def train_step(self, real_images, g_optimizer, c_optimizer, 
                   recon_weight=1.0, kl_weight=0.1, adv_weight=0.1, critic_iters=5):
        batch_size = real_images.size(0)
        real_images = real_images.to(device)
        
        # Train Critic multiple times with gradient penalty
        for _ in range(critic_iters):
            c_optimizer.zero_grad()
            
            # Real images
            real_score = self.critic(real_images)
            
            # Fake images from VAE
            with torch.no_grad():
                fake_images, _, _ = self.generator(real_images)
            fake_score = self.critic(fake_images)
            
            # Gradient penalty
            gp = self.critic.gradient_penalty(real_images, fake_images, device)
            
            c_loss = self.critic_loss(real_score, fake_score, gp)
            c_loss.backward()
            c_optimizer.step()
        
        # Train Generator (VAE)
        g_optimizer.zero_grad()
        
        # VAE forward pass
        recon, mu, logvar = self.generator(real_images)
        
        # Get critic score for reconstructed images
        critic_score = self.critic(recon)
        
        # Calculate losses
        recon_loss, kl_loss, adv_loss = self.vae_loss(real_images, recon, mu, logvar, critic_score)
        
        # Combined generator loss
        g_loss = recon_weight * recon_loss + kl_weight * kl_loss + adv_weight * adv_loss
        g_loss.backward()
        g_optimizer.step()
        
        return {
            'g_loss': g_loss.item(),
            'c_loss': c_loss.item(),
            'recon_loss': recon_loss.item(),
            'kl_loss': kl_loss.item(),
            'adv_loss': adv_loss.item(),
            'gp': gp.item()
        }


In [ ]:
class Trainer:
    """Training loop with visualization"""
    
    def __init__(self, model, device):
        self.model = model
        self.device = device
        self.history = defaultdict(list)
        self.logger = logging.getLogger('VAEGAN.Trainer')
        
    def train(self, dataloader, epochs=10, lr=1e-4):
        self.logger.info(f"Starting training for {epochs} epochs")
        
        # Use Adam for both optimizers with WGAN-GP
        g_optimizer = optim.Adam(self.model.generator.parameters(), lr=lr, betas=(0.0, 0.9))
        c_optimizer = optim.Adam(self.model.critic.parameters(), lr=lr, betas=(0.0, 0.9))
        
        for epoch in range(epochs):
            epoch_metrics = defaultdict(list)
            
            for batch_idx, (real_images, _) in enumerate(dataloader):
                metrics = self.model.train_step(real_images, g_optimizer, c_optimizer)
                
                for key, value in metrics.items():
                    epoch_metrics[key].append(value)
                    self.history[key].append(value)
                
                if batch_idx % 100 == 0:
                    self.logger.info(
                        f"Epoch {epoch} Batch {batch_idx}: "
                        f"G_loss={metrics['g_loss']:.4f} "
                        f"C_loss={metrics['c_loss']:.4f} "
                        f"Recon={metrics['recon_loss']:.4f} "
                        f"GP={metrics['gp']:.4f}"
                    )
            
            # Epoch summary
            avg_metrics = {k: np.mean(v) for k, v in epoch_metrics.items()}
            self.logger.info(
                f"Epoch {epoch} complete: "
                f"G_loss={avg_metrics['g_loss']:.4f} "
                f"C_loss={avg_metrics['c_loss']:.4f} "
                f"Recon={avg_metrics['recon_loss']:.4f} "
                f"KL={avg_metrics['kl_loss']:.4f} "
                f"Adv={avg_metrics['adv_loss']:.4f} "
                f"GP={avg_metrics['gp']:.4f}"
            )
            
            # Visualize reconstruction after each epoch
            self.plot_reconstruction(dataloader, epoch)
    
    def plot_reconstruction(self, dataloader, epoch):
        """Plot original vs reconstructed images"""
        self.model.eval()
        
        with torch.no_grad():
            # Get sample batch
            for real_images, _ in dataloader:
                break
            
            real_images = real_images[:8].to(self.device)
            recon_images, _, _ = self.model.generator(real_images)
            
            # Plot comparison
            fig, axes = plt.subplots(2, 8, figsize=(16, 4))
            
            for i in range(8):
                # Original
                img = real_images[i].cpu()
                if img.shape[0] == 1:
                    axes[0, i].imshow(img.squeeze(), cmap='gray', vmin=-1, vmax=1)
                else:
                    img = (img + 1) / 2  # Denormalize
                    axes[0, i].imshow(img.permute(1, 2, 0))
                axes[0, i].axis('off')
                
                # Reconstructed
                img = recon_images[i].cpu()
                if img.shape[0] == 1:
                    axes[1, i].imshow(img.squeeze(), cmap='gray', vmin=-1, vmax=1)
                else:
                    img = (img + 1) / 2  # Denormalize
                    axes[1, i].imshow(img.permute(1, 2, 0))
                axes[1, i].axis('off')
            
            axes[0, 0].set_ylabel('Original', fontsize=12)
            axes[1, 0].set_ylabel('Reconstructed', fontsize=12)
            plt.suptitle(f'Reconstruction Quality - Epoch {epoch}', fontsize=14)
            plt.tight_layout()
            plt.show()
        
        self.model.train()
    
    def plot_history(self):
        """Plot training loss history"""
        fig, axes = plt.subplots(2, 3, figsize=(15, 8))
        
        metrics = ['g_loss', 'c_loss', 'recon_loss', 'kl_loss', 'adv_loss', 'gp']
        
        for i, metric in enumerate(metrics):
            if metric in self.history:
                row, col = i // 3, i % 3
                axes[row, col].plot(self.history[metric])
                axes[row, col].set_title(metric.replace('_', ' ').title())
                axes[row, col].grid(True)
        
        # All subplots should be filled now with 6 metrics
        
        plt.suptitle('Training History', fontsize=16)
        plt.tight_layout()
        plt.show()


In [ ]:
if __name__ == "__main__":
    # Setup
    logger.info("Starting VAEGAN training")
    logger.info("=" * 50)
    
    # Create dataloader
    data_loader = DataLoaderModule(batch_size=64)
    
    # Load MNIST dataset  
    dataloader, channels, img_size = data_loader.load_mnist()
    
    # Create dynamic model with configurable latent_dim for MNIST
    latent_dim_mnist = 64  # Smaller for MNIST
    model = VAEGAN(input_channels=channels, latent_dim=latent_dim_mnist, img_size=64, lambda_gp=10)
    
    # Create trainer
    trainer = Trainer(model, device)
    
    # Train model
    logger.info("Training on MNIST")
    trainer.train(dataloader, epochs=10, lr=1e-4)
    
    # Plot results
    trainer.plot_history()
    
    logger.info("Training completed")
    
    # Train on CIFAR10 with larger latent_dim
    logger.info("Training on CIFAR10")
    dataloader_cifar, channels_cifar, _ = data_loader.load_cifar10()
    latent_dim_cifar = 128  # Larger for CIFAR10 (3 channels, more complex)
    model_cifar = VAEGAN(input_channels=channels_cifar, latent_dim=latent_dim_cifar, img_size=64, lambda_gp=10)
    trainer_cifar = Trainer(model_cifar, device)
    trainer_cifar.train(dataloader_cifar, epochs=5, lr=1e-4)
    trainer_cifar.plot_history()
